# グラフにする  
出力結果は画像ファイル
## 期間の区切り
https://news.yahoo.co.jp/byline/kutsunasatoshi/20210930-00259117
※第5波の終了は緊急事態宣言終了としている
|No|From|To|
|:-:|:-:|:-:|
|第1波|-|20200613|
|第2波|20200614|20201009|
|第3波|20201010|20210228|
|第4波|20210301|20210620|
|第5波|20210621|20210930|
|第6波|20211001|-|


In [1]:
import os
from WebScrapingTool import Base_UserFunction as uf
import json
import pandas as pd
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import json
import sys
import os
import numpy

In [2]:
def main():
    print("\n[Start]"  + uf.getNowTime() + '\n')

    #設定ファイルから必要な情報を取得する
    #タグ
    tag_debug = '[a]'
    tag_loadFolder = '[b]'
    tag_loadFileName = '[k]'
    tag_header = '[p]'
    
    isDebug = False
    _loadFolder = ''
    _loadFileName = ''
    _header = ''

    # カレントディレクトリ取得
    currentDir = '/'
    try:
        # Node-RED から呼び出し
        currentDir = os.path.dirname(__file__) + '/'
    except:
        # jupyterNotebook から呼び出し
        currentDir = os.path.dirname(os.path.abspath("__file__")) + '/'  
    print(currentDir)

    try:
        with open(currentDir + '_Setting.txt', mode='r') as f:
            lines = f.readlines()
            for l in lines:
                if l.startswith(tag_debug, 0, 3):
                    if (l.replace(tag_debug, '').rstrip()).lower() == 'true':
                        isDebug = True
                    else:
                        isDebug = False                    

                if l.startswith(tag_loadFolder, 0, 3):
                    _loadFolder = currentDir + l.replace(tag_loadFolder, '').rstrip()
             
                if l.startswith(tag_loadFileName, 0, 3):
                    _loadFileName = l.replace(tag_loadFileName, '').rstrip()
             
                if l.startswith(tag_header, 0, 3):
                    _header = l.replace(tag_header, '').rstrip()
             
    except:
        print('[!!!ERROR!!!] Read Setting.text')
        return        
    
    if len(_loadFolder) <= 0:
        print('[!!!ERROR!!!] Image data storage folder is None!')
        return  

    baseFile =_loadFolder + "/" + _loadFileName
    print(baseFile)
    headerList = _header.split(',')
    # ファイルを開く
    dfFile = pd.read_csv(baseFile, header=None, index_col=0, parse_dates=True)
    dfFile = dfFile.set_axis(headerList[1:], axis='columns')
    dfFile = dfFile.sort_index()

#    dfFile = pd.read_csv(baseFile, header=None)
#    dfFile = dfFile.set_axis(headerList, axis='columns')
#    dfFile.index = pd.to_datetime(dfFile['date'])
#    dfFile = dfFile.drop(columns = 'date')


    print(type(dfFile.index))
    print(type(dfFile.index[0]))    
#    print(dfFile.head(5))

    df = pd.DataFrame()

    # 新規感染者
    # under10 ~ 20
    print("Group1 : " + headerList[1] + " ~ " + headerList[3])
    df['Gr1'] = dfFile[headerList[1]] + dfFile[headerList[2]] + dfFile[headerList[3]]
    # 30 ~ 50
    print("Group2 : " + headerList[4] + " ~ " + headerList[6])
    df['Gr2'] = dfFile[headerList[4]] + dfFile[headerList[5]] + dfFile[headerList[6]]
    # 60 ~
    print("Group3 : " + headerList[7] + " ~ " + headerList[11])
    df['Gr3'] = dfFile[headerList[7]] + dfFile[headerList[8]] + dfFile[headerList[9]] + dfFile[headerList[10]] + dfFile[headerList[11]]

    # 重症者
    # under10 ~ 20
    print("GroupSeriouslyIll1 : " + headerList[13] + " ~ " + headerList[15])
    df['GrSI1'] = dfFile[headerList[13]] + dfFile[headerList[14]] + dfFile[headerList[15]]
    # 30 ~ 50
    print("GroupSeriouslyIll2 : " + headerList[16] + " ~ " + headerList[18])
    df['GrSI2'] = dfFile[headerList[16]] + dfFile[headerList[17]] + dfFile[headerList[18]]
    # 60 ~
    print("GroupSeriouslyIll3 : " + headerList[19] + " ~ " + headerList[23])
    df['GrSI3'] = dfFile[headerList[19]] + dfFile[headerList[20]] + dfFile[headerList[21]] + dfFile[headerList[22]] + dfFile[headerList[23]]

    df['PerSI1'] = df['GrSI1'] / df['Gr1']
    df['PerSI2'] = df['GrSI2'] / df['Gr2']
    df['PerSI3'] = df['GrSI3'] / df['Gr3']

    df['All'] = df['Gr1'] + df['Gr2'] + df['Gr3'] 
    df['AllSI'] = df['GrSI1'] + df['GrSI2'] + df['GrSI3'] 

    df = df.loc['2021-06-21' : '2100-12-31']


    dfOutput = pd.DataFrame()
    dfOutput['SI1'] = (df['PerSI1'] - df['PerSI1'].min()) / (df['PerSI1'].max() - df['PerSI1'].min())
    dfOutput['AllSI'] = (df['AllSI'] - df['AllSI'].min()) / (df['AllSI'].max() - df['AllSI'].min())
    dfOutput['All'] = (df['All'] - df['All'].min()) / (df['All'].max() - df['All'].min())
#    dfOutput['PerSI1'] = df['GrSI1'] / df['Gr1']
#    dfOutput['PerSI2'] = df['GrSI2'] / df['Gr2']
#    dfOutput['PerSI3'] = df['GrSI3'] / df['Gr3']
#    dfOutput = dfOutput.loc['2021-06-21' : '2100-12-31']
    print(dfOutput)

    plt.style.use('ggplot') 
    plt.figure()
    dfOutput.plot()
    plt.savefig('./Covid-19.png', dpi=120)
    plt.close('all')

    print("\n[ End ]"  + uf.getNowTime() + '\n')
    
    
if __name__ == '__main__':
    main()


[Start]20211013001655

d:\#WorkSpace\ReadCovid-19/
d:\#WorkSpace\ReadCovid-19/list/outputList.csv
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Group1 : under10 ~ 20
Group2 : 30 ~ 50
Group3 : 60 ~ 100
GroupSeriouslyIll1 : SeriouslyIll_under ~ SeriouslyIll_20
GroupSeriouslyIll2 : SeriouslyIll_30 ~ SeriouslyIll_50
GroupSeriouslyIll3 : SeriouslyIll_60 ~ SeriouslyIll_100
                 SI1     AllSI       All
0                                       
2021-06-21  0.000000  0.038462  0.032669
2021-06-22  0.000000  0.030769  0.067435
2021-06-23  0.000000  0.026923  0.099581
2021-06-24  0.000000  0.023077  0.091020
2021-06-25  0.000000  0.003846  0.089623
...              ...       ...       ...
2021-10-08  0.328042  0.119231  0.015549
2021-10-09  0.529915  0.119231  0.005765
2021-10-10  0.826667  0.115385  0.001922
2021-10-11  0.516667  0.096154  0.000000
2021-10-12  0.287037  0.069231  0.004892

[113 rows x 3 columns]

[ End ]20211